In [1]:
import re

from itertools import cycle, combinations, permutations
from collections import Counter, defaultdict, deque
from io import StringIO

def read_input(day, fn=str.strip):
    """
    Return a list of the input lines mapped by fn
    
    example: 
    >>> read_input('01', int)  # read input file, map all lines to int
    
    Inspired by Peter Norvig: https://github.com/norvig/pytudes
    
    """
    return list(map(fn, open(f'input\{day}.txt')))

def all_integers(s):
    """return all integers from a string"""
    return tuple(map(int, re.findall(r'-?\d+', s)))

# Day 1

In [2]:
def div3add2(x):
    return x // 3 - 2
assert div3add2(12) == 2

In [3]:
def total_fuel(m):
    fuel = 0
    while m > 0:
        #print(m, fuel)
        m = div3add2(m)
        if m > 0:
            fuel += m
    return fuel
assert total_fuel(1969) == 966

In [4]:
# part 1
sum(map(div3add2, read_input('01', int)))

3318604

In [5]:
# part 2

In [6]:
sum(map(total_fuel, read_input('01', int)))

4975039

# Day 2


In [7]:
def solve(ins): 
    ip = 0
    #while True:
    for _ in range(500):
        #print(ins)
        if ins[ip] == 99:
            return ins[0]

        opcode = ins[ip]
        a = ins[ins[ip+1]]
        b = ins[ins[ip+2]]
        res = ins[ip+3]
        #print(f"{opcode} {a} {b} {res}")
        if opcode == 1:
            ins[res] = a + b 
        elif opcode == 2:
            ins[res] = a * b
        else:
            return -1
            #assert False, "illegal instruction"
        ip += 4

testcase = list(all_integers("1,9,10,3,2,3,11,0,99,30,40,50")) 
testcase = list(all_integers("1,1,1,4,99,5,6,0,99"))

solve(testcase)

30

In [8]:
# part 1
opcodes = list(read_input('02', all_integers)[0])
opcodes[1] = 12
opcodes[2] = 2
solve(opcodes)

10566835

In [9]:
TARGET = 19690720
for noun, verb in permutations(range(100), 2):
    opcodes = list(read_input('02', all_integers)[0])  # too lazy to deepcopy
    opcodes[1] = noun
    opcodes[2] = verb
    res = solve(opcodes)
    if res == TARGET:
        print(100*noun+verb)
        break


2347


# Day 3




In [10]:
testcase1 = ["R8,U5,L5,D3", "U7,R6,D4,L4"]

In [11]:
next_pos = {'U': lambda x,y: (x, y + 1),
            'D': lambda x,y: (x, y - 1), 
            'L': lambda x,y: (x - 1, y),
            'R': lambda x,y: (x + 1, y)}

In [12]:
def parse(lines):
    return [s.split(',') for s in lines]

def create_path(cmds):  
    visited = list()
    pos = (0, 0)
    for cmd in cmds:
        d = cmd[0]
        n = int(cmd[1:])
        for _ in range(n):
            pos = next_pos[d](*pos)
            visited.append(pos)
    return visited

def manhattan_distance(X1, Y1, X2 = 0, Y2 = 0):
    return abs(Y2 - Y1) + abs(X2 - X1)

def solve(input_string):
    cmds = parse(input_string)
    paths = [set(create_path(cmd)) for cmd in cmds]
    sols = set.intersection(*paths)
    return min([manhattan_distance(*t) for t in sols])

solve(testcase1)


6

In [13]:
# part 1
solve(read_input('03'))


308

In [14]:
def solve_partB(input_string):
    cmds = parse(input_string)
    paths = [create_path(cmd) for cmd in cmds] 
    set_paths = [set(path) for path in paths]
    intersections = set.intersection(*set_paths)
    sol = map(sum, [[path.index(intersection) + 1 for path in paths] for intersection in intersections])
    return min(sol)

solve_partB(testcase1)

30

In [15]:
solve_partB(read_input('03'))


12934

# Day 4

In [16]:
def is_valid(s):
    if sorted(s) != list(s):
        return False
    if max(Counter(s).values()) == 1:
        return False
    return True

is_valid('111111'), is_valid('223450'), is_valid('123789')    

(True, False, False)

In [17]:
START = 123257
END = 647015

In [18]:
sum((is_valid(str(i)) for i in range(START, END+1)))

2220

In [19]:
# part 2
def is_valid(s):
    if sorted(s) != list(s):
        return False
    if 2 not in Counter(s).values():
        return False
    return True

is_valid('112233'), is_valid('123444'), is_valid('111122')    

(True, False, True)

In [20]:
sum((is_valid(str(i)) for i in range(START, END+1)))

1515

In [21]:
# part A and codegolf:
def is_sorted(s):
    return sorted(s) == list(s)

n = [str(i) for i in range(START, END+1)]
sorted_n = [s for s in n if is_sorted(s)]
sum((max(Counter(s).values()) > 1 for s in sorted_n)), sum((2 in Counter(s).values() for s in sorted_n))

(2220, 1515)

# Day 5

In [22]:
def decode(instruction):
    opcode = instruction % 100
    A = instruction % 100000 // 10000
    B = instruction % 10000 // 1000
    C = instruction % 1000 // 100
    return A, B, C, opcode
decode(12345), decode(1), decode(1002)

((1, 2, 3, 45), (0, 0, 0, 1), (0, 1, 0, 2))

In [23]:
def get_input():
    while True:
        print('INPUT!')
        yield 0

In [24]:
from copy import deepcopy

class IntCode(object):
    
    ip = 0
    output = None
    
    def __init__(self, program, input=None, debug=False):
        self.mem = deepcopy(program)
        self.input = iter(input)
        self.debug = debug
    
    def get_opcode(self):
        self.ip += 1
        instruction = self.mem[self.ip]
        if self.debug:
            print(f'ins = {instruction} {self.mem[self.ip:self.ip+4]}')
        opcode = instruction % 100
        A = instruction % 100000 // 10000
        B = instruction % 10000 // 1000
        C = instruction % 1000 // 100
        self.modes = [A, B, C]
        return opcode
    
    def load(self):
        self.ip += 1
        value = self.mem[self.ip]
        mode = self.modes.pop()
        #print(f'load: {value} {mode} {self.modes}')
        if mode == 0:
            #print(f'load {self.mem[value]}')
            return self.mem[value]
        if mode == 1:
            return value
        assert False, f'wrong mode for load: {mode}'

    def store(self, value):
        self.ip += 1
        mode = self.modes.pop()
        #print(f'store {self.mem[self.ip]} {mode} {self.modes}')
        if mode == 0:
            self.mem[self.mem[self.ip]] = value
            #print(self.mem)
            return
        assert False, f'wrong mode for store: {mode}'
        
    def run(self, ip = 0):
        self.ip = ip - 1
        while True:
            opcode = self.get_opcode()
            if opcode == 1:
                self.store(self.load() + self.load())
            elif opcode == 2:
                self.store(self.load() * self.load())
            elif opcode == 3:
                self.store(next(self.input))  
            elif opcode == 4:
                self.output = self.load()
                #print(f'>>> output: {self.output}')
            elif opcode == 5:  # jmp if true
                if self.load():
                    self.ip = self.load() - 1
                else:
                    self.ip += 1
            elif opcode == 6:  # jmp if false
                if not self.load():
                    self.ip = self.load() - 1
                else:
                    self.ip += 1
            elif opcode == 7:  # less than
                val = 1 if self.load() < self.load() else 0
                self.store(val)
            elif opcode == 8:  # equal
                val = 1 if self.load() == self.load() else 0
                self.store(val)    
            elif opcode == 99:
                return self.output
            else:
                assert False, f'fail @ ip={self.ip}'  
    

In [25]:
program = [3,3,1107,-1,8,3,4,3,99]
IntCode(program, input=[-15], debug=True).run()

ins = 3 [3, 3, 1107, -1]
ins = 1107 [1107, -15, 8, 3]
ins = 4 [4, 3, 99]
ins = 99 [99]


1

In [26]:
IntCode(program, input=[-15], debug=True).run()

ins = 3 [3, 3, 1107, -1]
ins = 1107 [1107, -15, 8, 3]
ins = 4 [4, 3, 99]
ins = 99 [99]


1

In [27]:
program = [3,21,1008,21,8,20,1005,20,22,107,8,21,20,1006,20,31,
1106,0,36,98,0,0,1002,21,125,20,4,20,1105,1,46,104,
999,1105,1,46,1101,1000,1,20,4,20,1105,1,46,98,99]
IntCode(program, input=[18], debug=True).run()

ins = 3 [3, 21, 1008, 21]
ins = 1008 [1008, 21, 8, 20]
ins = 1005 [1005, 20, 22, 107]
ins = 107 [107, 8, 21, 20]
ins = 1006 [1006, 20, 31, 1106]
ins = 1106 [1106, 0, 36, 98]
ins = 1101 [1101, 1000, 1, 20]
ins = 4 [4, 20, 1105, 1]
ins = 1105 [1105, 1, 46, 98]
ins = 99 [99]


1001

In [28]:
#part A
program = list(read_input('05', all_integers)[0])
IntCode(program, input=[1]).run()

13818007

In [29]:
# part B
program = list(read_input('05', all_integers)[0])
IntCode(program, input=[5]).run()

3176266

# Day 6

In [30]:
testcase1 = """COM)B
B)C
C)D
D)E
E)F
B)G
G)H
D)I
E)J
J)K
K)L"""

In [31]:
def parse(line):
    return line.strip('\n').split(')')

with StringIO(testcase1) as f:
    testcase = list(map(parse, f))
testcase

[['COM', 'B'],
 ['B', 'C'],
 ['C', 'D'],
 ['D', 'E'],
 ['E', 'F'],
 ['B', 'G'],
 ['G', 'H'],
 ['D', 'I'],
 ['E', 'J'],
 ['J', 'K'],
 ['K', 'L']]

In [32]:
tree = {orbit: planet for planet, orbit in testcase}
tree

{'B': 'COM',
 'C': 'B',
 'D': 'C',
 'E': 'D',
 'F': 'E',
 'G': 'B',
 'H': 'G',
 'I': 'D',
 'J': 'E',
 'K': 'J',
 'L': 'K'}

In [33]:
c = 0
for orbit in tree.keys():
    p = tree.get(orbit, None)
    while p:
        c += 1
        p = tree.get(p, None)
print(c)

42


In [34]:
input = [s.split(')') for s in read_input('06')]
tree = {orbit: planet for planet, orbit in input}
c = 0
for orbit in tree.keys():
    p = tree.get(orbit, None)
    while p:
        c += 1
        p = tree.get(p, None)
print(c)


135690


In [35]:
testcase2 = """COM)B
B)C
C)D
D)E
E)F
B)G
G)H
D)I
E)J
J)K
K)L
K)YOU
I)SAN"""

In [36]:
def parse(line):
    return line.strip('\n').split(')')

with StringIO(testcase2) as f:
    testcase = list(map(parse, f))
testcase

[['COM', 'B'],
 ['B', 'C'],
 ['C', 'D'],
 ['D', 'E'],
 ['E', 'F'],
 ['B', 'G'],
 ['G', 'H'],
 ['D', 'I'],
 ['E', 'J'],
 ['J', 'K'],
 ['K', 'L'],
 ['K', 'YOU'],
 ['I', 'SAN']]

In [37]:
tree = defaultdict(list)
for planet, orbit in testcase:
    tree[orbit].append(planet)
    tree[planet].append(orbit)
tree

defaultdict(list,
            {'B': ['COM', 'C', 'G'],
             'COM': ['B'],
             'C': ['B', 'D'],
             'D': ['C', 'E', 'I'],
             'E': ['D', 'F', 'J'],
             'F': ['E'],
             'G': ['B', 'H'],
             'H': ['G'],
             'I': ['D', 'SAN'],
             'J': ['E', 'K'],
             'K': ['J', 'L', 'YOU'],
             'L': ['K'],
             'YOU': ['K'],
             'SAN': ['I']})

In [38]:
def bfs(start, goal, tree):
    visited, queue = set(), [(0, start)]
    while queue:
        moves, pos = queue.pop(0)
        #print('queue:', queue)
        if pos not in visited:
            visited.add(pos)
            if pos == goal:
                return moves
            queue.extend([(moves+1, planet) for planet in tree[pos]])


In [39]:
bfs(tree['YOU'][0], tree['SAN'][0], tree)

4

In [40]:
tree = defaultdict(list)
for planet, orbit in input:
    tree[orbit].append(planet)
    tree[planet].append(orbit)

In [41]:
bfs(tree['YOU'][0], tree['SAN'][0], tree)

298

# Day 7

In [42]:
testcase = [3,15,3,16,1002,16,10,16,1,16,15,15,4,15,99,0,0]

In [43]:
def calc(program, phase):
    res = 0
    for p in phase:
        res = IntCode(program, input=[p, res]).run()
    
    return res

calc(testcase, [4, 3, 2, 1, 0])

43210

In [44]:
testcase = [3,31,3,32,1002,32,10,32,1001,31,-2,31,1007,31,0,33,1002,33,7,33,1,33,31,31,1,32,31,31,4,31,99,0,0,0]
calc(testcase, [1, 2, 3, 4, 0])

60125

In [45]:
def solve(program):
    m = 0
    for p1, p2, p3, p4, p5 in permutations(range(5)):
        res = calc(program, [p1, p2, p3, p4, p5])
        if res > m:
            #print('found:', p1, p2, p3, p4, p5)
            m = res
            sol = [p1, p2, p3, p4, p5]
    return m, sol
    
solve(testcase)

(65210, [1, 0, 4, 3, 2])

In [46]:
program = list(read_input('07', all_integers)[0])
solve(program)

(18812, [2, 3, 0, 4, 1])

In [47]:
#part B

class IntCode(object):
    
    ip = 0
    output = None
    halted = False
    
    def __init__(self, program, input=None, debug=False):
        self.mem = deepcopy(program)
        self.input = input
        self.debug = debug
    
    def get_opcode(self):
        self.ip += 1
        instruction = self.mem[self.ip]
        if self.debug:
            print(f'ins = {instruction} {self.mem[self.ip:self.ip+4]}')
        opcode = instruction % 100
        A = instruction % 100000 // 10000
        B = instruction % 10000 // 1000
        C = instruction % 1000 // 100
        self.modes = [A, B, C]
        return opcode
    
    def load(self):
        self.ip += 1
        value = self.mem[self.ip]
        mode = self.modes.pop()
        #print(f'load: {value} {mode} {self.modes}')
        if mode == 0:
            #print(f'load {self.mem[value]}')
            return self.mem[value]
        if mode == 1:
            return value
        assert False, f'wrong mode for load: {mode}'

    def store(self, value):
        self.ip += 1
        mode = self.modes.pop()
        #print(f'store {self.mem[self.ip]} {mode} {self.modes}')
        if mode == 0:
            self.mem[self.mem[self.ip]] = value
            #print(self.mem)
            return
        assert False, f'wrong mode for store: {mode}'
        
    def run(self):
        self.ip -= 1
        while True:
            opcode = self.get_opcode()
            if opcode == 1:
                self.store(self.load() + self.load())
            elif opcode == 2:
                self.store(self.load() * self.load())
            elif opcode == 3:
                try:
                    val = self.input.pop(0)
                except IndexError:
                    #print('Waiting for input')
                    return self.output
                #print(f'read: {val}')
                self.store(val)  
            elif opcode == 4:
                self.output = self.load()
                #print(f'>>> output: {self.output}')
            elif opcode == 5:  # jmp if true
                if self.load():
                    self.ip = self.load() - 1
                else:
                    self.ip += 1
            elif opcode == 6:  # jmp if false
                if not self.load():
                    self.ip = self.load() - 1
                else:
                    self.ip += 1
            elif opcode == 7:  # less than
                val = 1 if self.load() < self.load() else 0
                self.store(val)
            elif opcode == 8:  # equal
                val = 1 if self.load() == self.load() else 0
                self.store(val)    
            elif opcode == 99:
                self.halted = True
                return self.output
            else:
                assert False, f'fail @ ip={self.ip}'  
    

testcase = [3,26,1001,26,-4,26,3,27,1002,27,2,27,1,27,26,27,4,27,1001,28,-1,28,1005,28,6,99,0,0,5]
i = IntCode(testcase, input=[5, 0], debug=True)
i.run()

ins = 3 [3, 26, 1001, 26]
ins = 1001 [1001, 26, -4, 26]
ins = 3 [3, 27, 1002, 27]
ins = 1002 [1002, 27, 2, 27]
ins = 1 [1, 27, 26, 27]
ins = 4 [4, 27, 1001, 28]
ins = 1001 [1001, 28, -1, 28]
ins = 1005 [1005, 28, 6, 99]
ins = 3 [3, 27, 1002, 27]


1

In [48]:
i.input = [15]
i.run()

ins = 3 [3, 27, 1002, 27]
ins = 1002 [1002, 27, 2, 27]
ins = 1 [1, 27, 26, 27]
ins = 4 [4, 27, 1001, 28]
ins = 1001 [1001, 28, -1, 28]
ins = 1005 [1005, 28, 6, 99]
ins = 3 [3, 27, 1002, 27]


31

In [49]:
def calc(program, phase):
    res = 0
    amps = [IntCode(program, input=[p]) for p in phase]
    i = 0
    halted = 5 * [False]
    while True:
        i = i % len(amps)
        
        #print(f'----------> amp: {i}')
        amps[i].input.append(res)
        res = amps[i].run()
        if amps[i].halted:
            halted[i] = True
            #print('halted', halted)
            if False not in halted:
                break
        i += 1
    return res

calc(testcase, [9, 8, 7, 6, 5])

139629729

In [50]:
testcase = [3,52,1001,52,-5,52,3,53,1,52,56,54,1007,54,5,55,1005,55,26,1001,54,-5,54,1105,1,12,1,53,54,53,1008,54,0,55,1001,55,1,55,2,53,55,53,4,
53,1001,56,-1,56,1005,56,6,99,0,0,0,0,10]

In [51]:
calc(testcase, [9,7,8,5,6])

18216

In [52]:
def solve(program):
    m = 0
    for p1, p2, p3, p4, p5 in permutations(range(5, 10)):
        res = calc(program, [p1, p2, p3, p4, p5])
        if res > m:
            print('found:', p1, p2, p3, p4, p5)
            m = res
            sol = [p1, p2, p3, p4, p5]
    return m, sol
    
solve(program)

found: 5 6 7 8 9
found: 5 6 7 9 8
found: 5 6 9 7 8
found: 5 6 9 8 7
found: 6 5 7 9 8
found: 6 5 9 7 8
found: 6 5 9 8 7
found: 6 7 9 5 8
found: 6 7 9 8 5
found: 6 9 5 7 8
found: 6 9 5 8 7
found: 6 9 7 5 8
found: 6 9 7 8 5
found: 6 9 8 7 5


(25534964, [6, 9, 8, 7, 5])

# Day 8

In [10]:
input = list(map(int, list(str(read_input('08')[0]))))
row_size = 25
rows = 6

In [16]:
N = rows * row_size
c_min = N
for idx in range(0, len(input), N):
    layer = input[idx:idx+N]
    zeros = layer.count(0)
    if zeros < c_min:
        res = layer.count(1) * layer.count(2)
        c_min = zeros
        
res

2193

In [17]:
#part B

In [18]:
image = N*[2]
for idx in range(0, len(input), N):
    layer = input[idx:idx+N]
    for idx, color in enumerate(layer):
        if image[idx] == 2:
            image[idx] = color 

color_map = {0: '.', 1: '#', 2: '!'}

def print_image(image):
    for idx in range(0, len(image), row_size):
        print(''.join([color_map[c] for c in image[idx:idx+row_size]]))
    
print_image(image)

#...#####.#..#.####.####.
#...##....#..#.#....#....
.#.#.###..####.###..###..
..#..#....#..#.#....#....
..#..#....#..#.#....#....
..#..####.#..#.####.#....
